In [1]:
%matplotlib widget
# Load packages and settings
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 50)
import seaborn as sns


import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (12,8)
plt.rcParams["image.cmap"] = "tab10"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab10.colors)
fs_label = 16
parameters = {
                'axes.labelsize': fs_label,
                'axes.titlesize': fs_label+4,
                'xtick.labelsize': fs_label,
                'ytick.labelsize': fs_label, 
                'legend.fontsize': fs_label, 
                'lines.markersize': 10,
                'lines.linewidth': 3
             }
plt.rcParams.update(parameters)

from matplotlib import cm # Colormaps
import matplotlib.colors as colors
# cmap = plt.cm.get_cmap('Dark2',len(ageGroups))
# cmap = colors.LinearSegmentedColormap.from_list("", ["red","violet","blue","green"],len(allAge))
# cmap = plt.cm.get_cmap('turbo',len(allAge))

import locale
import matplotlib.dates as mdates
locale.setlocale(locale.LC_TIME,"Danish")
# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))
# ax1.spines['top'].set_visible(False) 

import os
# import csv
import math

from datetime import date


saveFigures = True
# saveFigures = False
print('saveFigures is set to: '+str(saveFigures))

print('Done loading packages')

# Define running mean functions
def rnMean(data,meanWidth):
    return np.convolve(data, np.ones(meanWidth)/meanWidth, mode='valid')
def rnTime(t,meanWidth):
    return t[math.floor(meanWidth/2):-math.ceil(meanWidth/2)+1]

saveFigures is set to: True
Done loading packages


In [2]:
# Define paths
rootdir_data = os.getcwd() +"\\..\\DanskeData\\" 

path_data = rootdir_data + "ssi_data\\"
path_dash = rootdir_data + "ssi_dashboard\\"
path_vacc = rootdir_data + "ssi_vacc\\"
path_figs = os.getcwd() +"\\..\\Figures\\" 

path_figs = path_figs+"Testcenter\\"

# Get data

In [3]:
# Load dataframe with testcenter names
curSubDir = list(os.walk(path_dash))[0][1][-1]
curdir = path_dash + curSubDir
curfilepath = curdir+'/Testcenter_DB/01_pcr_positiv_procent_pr_test_center_de_seneste_7_dage.csv'

df_main = pd.read_csv(curfilepath,encoding='latin1',delimiter = ';')
df_main.head()
df_main['Testcenter navn'].unique()

array(['Aabenraa/Rødekro', 'Aalborg', 'Aarhus andre testcentre', 'Aars',
       'Albertslund', 'Allerød', 'Aulum', 'Ballerup', 'Billund Lufthavn',
       'Skanderborg', 'Bjerringbro', 'Bornholm - Rønne',
       'Brande Hallerne', 'Brovst', 'Brædstrup Borgerhus', 'Brøndby',
       'Brønderslev', 'Dragør', 'Ebeltoft/Rønde', 'Esbjerg', 'Farum',
       'Fredericia', 'Frederiksberg', 'Frederikshavn', 'Frederikssund',
       'Gammel Aldi Viby', 'Gentofte', 'Gladsaxe', 'Glostrup', 'Grenaa',
       'Gribskov', 'Grindsted', 'Haderslev', 'Halsnæs',
       'Haslev - Vesterled Hallen', 'Hedehusene', 'Hedensted Gruppen',
       'Helsingør', 'Herlev', 'Hillerød', 'Hillerød - Grønnevang',
       'Humlebæk', 'Hvalsø', 'Hvidovre', 'Ikast gl. Vestre Skole',
       'Ishøj', 'Kirke Hyllinge', 'Kolding', 'København',
       'København - Amager', 'Københavns Lufthavn', 'Køge', 'Lemvig',
       'Lyngby', 'Læsø', 'Middelfart', 'Måløv', 'Nakskov', 'Nyborg',
       'Nykøbing F', 'Nykøbing Mors', 'Næstved', 'Ode

In [4]:
curTestName = 'København'
# curTestName = 'Aarhus andre testcentre'
# curTestName = 'Aalborg'
curYdernummer = df_main[df_main['Testcenter navn'] == curTestName].Ydernummer.values[0]
curYdernummer
strName = f'Testcenter: {curTestName}, nr: {curYdernummer}'
strName

'Testcenter: København, nr: 905373'

In [5]:
# df_main

In [6]:
allSubDirs = list(os.walk(path_dash))[0][1]

# df = pd.DataFrame()

allDates = []
allVals = []
allPos = []

for curSubDir in allSubDirs:
    curdir = path_dash + curSubDir
    curfilepath = curdir+'/Testcenter_DB/05_pcr_testede_personer_pr_test_center_pr_alders_grp_de_seneste_7_dage.csv'
    curfilepath = curdir+'/Testcenter_DB/07_pcr_testede_personer_pr_test_center_pr_fnkt_alder_de_seneste_7_dage.csv'
    # curfilepath = curdir+'/Testcenter_DB/01_pcr_positiv_procent_pr_test_center_de_seneste_7_dage.csv'

    # Check if file was included at the time.
    if os.path.isfile(curfilepath):
        curdf = pd.read_csv(curfilepath,encoding='latin1',delimiter = ';')
        
        allDates.append(curdir[-10:])
        allVals.append(curdf[curdf.Ydernummer == curYdernummer]['PCR testede personer de seneste 7 dage'].values)
        
 
df = pd.DataFrame(allVals,columns=curdf.Aldersgruppe.unique())
df['Date'] = pd.to_datetime(allDates)


In [7]:
curdf

,Ydernummer,Aldersgruppe,PCR testede personer de seneste 7 dage
0,905331,00-02,91
1,905331,03-05,203
2,905331,06-11,378
3,905331,12-15,330
4,905331,16-19,317
5,905331,20-39,3091
6,905331,40-64,2657
7,905331,65-79,833
8,905331,80+,174
9,905370,00-02,15


In [8]:
df

,00-02,03-05,06-11,12-15,16-19,20-39,40-64,65-79,80+,Date
0,41,303,796,470,377,5174,3292,437,33,2021-09-22
1,41,290,755,460,349,5012,3258,419,34,2021-09-23
2,34,274,688,421,323,4744,3139,417,32,2021-09-24
3,27,258,709,322,283,4448,3110,378,26,2021-09-27
4,28,262,715,307,290,4353,3076,382,26,2021-09-28
5,28,281,767,314,291,4324,3090,412,20,2021-09-29
6,23,272,752,300,275,4122,3025,420,19,2021-09-30
7,30,272,760,287,280,4268,3045,410,23,2021-10-01
8,46,278,840,363,297,4317,3151,451,29,2021-10-04
9,43,269,885,400,273,4291,3204,466,28,2021-10-05


# Make a weekly figure

In [9]:
weeksBack = 20
datesToPlot = np.datetime64('2022-02-14')- [np.timedelta64(x,'D') for x in np.arange(0,7*weeksBack,7)]

thisdf = df[df.Date.isin(datesToPlot)]
thisdf 

allAges = df.columns[:-1]

# # colsToUse = [0:5,5]

fig,ax1 = plt.subplots()

ax1.plot(thisdf.Date,thisdf.iloc[:,0:5].sum(axis=1),'.-',label='0-19',)
ax1.plot(thisdf.Date,thisdf.iloc[:,5],'.-',label=allAges[5],)
ax1.plot(thisdf.Date,thisdf.iloc[:,6],'.-',label=allAges[6],)
ax1.plot(thisdf.Date,thisdf.iloc[:,7:9].sum(axis=1),'.-',label='65+',)

ax1.legend()
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%b'))
ax1.set_ylim(bottom=0)

ax1.set_xlim(left=np.min(thisdf.Date))

ax1.grid()

fig.suptitle(strName,fontsize=20)

fig.tight_layout()

if saveFigures:
    fig.savefig(path_figs+f'AntalTestede_{curTestName}')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
weeksBack = 20
datesToPlot = np.datetime64('2022-02-14')- [np.timedelta64(x,'D') for x in np.arange(0,7*weeksBack,7)]

thisdf = df[df.Date.isin(datesToPlot)]
thisdf 

allAges = df.columns[:-1]

# # colsToUse = [0:5,5]

fig,ax1 = plt.subplots()

# ax1.plot(thisdf.Date,thisdf.iloc[:,0:5].sum(axis=1),'.-',label='0-19',)
# ax1.plot(thisdf.Date,thisdf.iloc[:,5],'.-',label=allAges[5],)
# ax1.plot(thisdf.Date,thisdf.iloc[:,6],'.-',label=allAges[6],)
ax1.plot(thisdf.Date,thisdf.iloc[:,7:9].sum(axis=1),'r.-',label='65+',)

ax1.legend()
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%b'))
ax1.set_ylim(bottom=0)

ax1.set_xlim(left=np.min(thisdf.Date))

ax1.grid()

fig.suptitle(strName,fontsize=20)

fig.tight_layout()

if saveFigures:
    fig.savefig(path_figs+f'AntalTestede_{curTestName}_65+')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
# # curdf[curdf.Ydernummer == curYdernummer]['PCR testede personer de seneste 7 dage'].values
# # df_main
# # curdir[-10:] 
# allDates

# curdf.Aldersgruppe.unique() 
df_main.sort_values(by='Testede personer de seneste 7 dage')

,Ydernummer,Breddegrader,Længdegrader,Testcenter navn,Testede personer de seneste 7 dage,Bekræftede tilfælde de seneste 7 dage,Positiv procenten de seneste 7 dage
2,905921,56.179517,10.167339,Aarhus andre testcentre,23,6,26.1
6,906468,56.265671,8.797120,Aulum,268,93,34.7
40,906266,55.935395,12.341552,Hillerød - Grønnevang,287,86,30.0
54,905779,57.254032,11.001293,Læsø,301,75,24.9
91,905917,56.277798,8.319880,Vestjysk Fritidscenter Ulfborg,352,135,38.4
95,906267,55.940605,12.266168,Ålholm,382,136,35.6
56,906212,55.749767,12.334563,Måløv,455,147,32.3
69,906206,55.716861,12.399646,Skovlunde,465,129,27.7
42,905927,55.592095,11.859938,Hvalsø,630,225,35.7
71,905896,54.992309,12.289648,Stege,918,319,34.7
